In [128]:
import pandas as pd
import unicodedata
import json
import json
from datetime import date


In [129]:
# Función para normalizar nombres de canciones (quita espacios extra y pasa a minúsculas)
def normalize(song):
    song = song.strip().lower()
    song = ''.join(c for c in unicodedata.normalize('NFKD', song) if not unicodedata.combining(c) or c == 'ñ')
    return song


In [130]:
# Cargar los archivos JSON
with open(r'C:\Users\Yo\Desktop\setlist\SetlistFM-Setlist-Evolution\album_data\Los Piojos.json', 'r', encoding='utf-8') as f:
    albums_data = json.load(f)


with open('Setlists_Piojos.json', 'r', encoding='utf-8') as f:
    setlists_data = json.load(f)

In [131]:

# Crear diccionarios para mapear cada canción a su álbum y a su color
song_to_album = {}
song_to_color = {}
for album in albums_data.get("albums", []):
    album_title = album.get("title", "Ineditos")
    color = album.get("color", "gray")
    for track in album.get("tracks", []):
        norm_track = normalize(track)
        # Si la canción aparece en varios álbumes, se asigna el primero encontrado
        if norm_track not in song_to_album:
            song_to_album[norm_track] = album_title
            song_to_color[norm_track] = color

# Crear un diccionario que mapea cada álbum a su color en formato HEX
album_to_color = {}
for album in albums_data.get("albums", []):
    album_to_color[album.get("title", "Ineditos")] = album.get("color", "#39df00")

In [132]:

df_popularidad = pd.read_excel(r'C:\Users\Yo\Desktop\setlist\SetlistFM-Setlist-Evolution\canciones_popularidad.xlsx')


# Ordenar los setlists por fecha (usando year, month, day)
setlists_data.sort(key=lambda s: date(s["year"], s["month"], s["day"]))

# Procesar cada setlist y contar acumulativamente
records = []
cumulative_counts = {}

for setlist in setlists_data:
    # Crear fecha en formato ISO
    current_date = date(setlist["year"], setlist["month"], setlist["day"]).isoformat()
    place = setlist["venue"]
    # Procesar la lista de canciones, separando por "/" si es necesario
    canciones_raw = setlist.get("songs", [])
    canciones = []
    for item in canciones_raw:
        for part in item.split('/'):
            if part.strip():
                canciones.append(part.strip())
    
    # Actualizar los conteos acumulados
    for song in canciones:
        norm_song = normalize(song)
        cumulative_counts[norm_song] = cumulative_counts.get(norm_song, 0) + 1
    # Registrar el estado acumulado hasta la fecha actual
    for song, count in cumulative_counts.items():
        album = song_to_album.get(song, "Ineditos")
        color = song_to_color.get(song, "#000000")
        records.append({
            "fecha": current_date,
            "lugar": place,
            "canción": song.title(),  # Capitalizamos para visualización
            "álbum": album,
            "color": color,
            "veces_tocada": count
        })

# Crear DataFrame y agrupar para quedarnos con el último conteo de cada fecha
df_vuelta_total = pd.DataFrame(records)


df_vuelta_total[df_vuelta_total['fecha'] == '2025-05-25'].head()

# Asume que song_to_album y song_to_color ya están definidos


all_records = []

for setlist in setlists_data:
    current_date = date(setlist["year"], setlist["month"], setlist["day"]).isoformat()
    place = setlist["venue"]
    canciones_raw = setlist.get("songs", [])
    canciones = []
    for item in canciones_raw:
        for part in item.split('/'):
            if part.strip():
                canciones.append(part.strip())
    for song in canciones:
        norm_song = normalize(song)
        album = song_to_album.get(norm_song, "Ineditos")
        color = song_to_color.get(norm_song, "#000000")
        all_records.append({
            "fecha": current_date,
            "lugar": place,
            "canción": song.title(),
            "álbum": album,
            "color": color  # Solo cuenta 1 para este concierto
        })

# Super DataFrame con todas las fechas y canciones (sin acumulativo)
df_super = pd.DataFrame(all_records)

df_super

,fecha,lugar,canción,álbum,color
0,1991-09-21,"Boa Vista, Buenos Aires, Argentina",Blues Del Gato Sarnoso,Ineditos,#d57f5c
1,1991-09-21,"Boa Vista, Buenos Aires, Argentina",Los Mocosos,Chac Tu Chac,#1e2830
2,1991-09-21,"Boa Vista, Buenos Aires, Argentina",Blues Del Traje Gris,Chac Tu Chac,#1e2830
3,1991-09-21,"Boa Vista, Buenos Aires, Argentina",Chac Tu Chac,Chac Tu Chac,#1e2830
4,1991-09-21,"Boa Vista, Buenos Aires, Argentina",Babilonia,Ay Ay Ay,#c41616
...,...,...,...,...,...
4100,2025-05-25,"Parque de la Ciudad, Buenos Aires, Argentina",El Farolito,3er Arco,#ead936
4101,2025-05-25,"Parque de la Ciudad, Buenos Aires, Argentina",El Balneario De Los Doctores Crotos,Azul,#0047a5
4102,2025-05-25,"Parque de la Ciudad, Buenos Aires, Argentina",Cruel,Chac Tu Chac,#1e2830
4103,2025-05-25,"Parque de la Ciudad, Buenos Aires, Argentina",Finale,Azul,#0047a5


In [133]:
df_super['cancion_norm'] = df_super['canción'].apply(normalize)
df_popularidad['cancion_norm'] = df_popularidad['canción'].apply(normalize)

In [134]:

# 2. Une la popularidad a cada canción tocada en cada concierto
df_super = df_super.merge(
    df_popularidad[['cancion_norm', 'indice_popularidad', 'indice_popularidad_norm']],
    on='cancion_norm',
    how='left'
)

df_super

,fecha,lugar,canción,álbum,color,cancion_norm,indice_popularidad,indice_popularidad_norm
0,1991-09-21,"Boa Vista, Buenos Aires, Argentina",Blues Del Gato Sarnoso,Ineditos,#d57f5c,blues del gato sarnoso,-0.787491,0.061883
1,1991-09-21,"Boa Vista, Buenos Aires, Argentina",Los Mocosos,Chac Tu Chac,#1e2830,los mocosos,-0.108193,0.235819
2,1991-09-21,"Boa Vista, Buenos Aires, Argentina",Blues Del Traje Gris,Chac Tu Chac,#1e2830,blues del traje gris,-0.489090,0.138290
3,1991-09-21,"Boa Vista, Buenos Aires, Argentina",Chac Tu Chac,Chac Tu Chac,#1e2830,chac tu chac,0.019795,0.268590
4,1991-09-21,"Boa Vista, Buenos Aires, Argentina",Babilonia,Ay Ay Ay,#c41616,babilonia,0.204522,0.315890
...,...,...,...,...,...,...,...,...
4100,2025-05-25,"Parque de la Ciudad, Buenos Aires, Argentina",El Farolito,3er Arco,#ead936,el farolito,2.600115,0.929285
4101,2025-05-25,"Parque de la Ciudad, Buenos Aires, Argentina",El Balneario De Los Doctores Crotos,Azul,#0047a5,el balneario de los doctores crotos,0.658090,0.432027
4102,2025-05-25,"Parque de la Ciudad, Buenos Aires, Argentina",Cruel,Chac Tu Chac,#1e2830,cruel,0.159382,0.304332
4103,2025-05-25,"Parque de la Ciudad, Buenos Aires, Argentina",Finale,Azul,#0047a5,finale,0.146156,0.300945


In [135]:
# 3. Calcula el promedio de popularidad por concierto (por fecha)
df_conciertos = df_super.groupby('fecha')['indice_popularidad_norm'].mean().reset_index()
df_conciertos = df_conciertos.sort_values('indice_popularidad_norm', ascending=False)


# 4. Ahora puedes ver los conciertos más "mainstream" y más "alternativos"
print("Recitales ordenados de mayor a menor popularidad en promedio:")
print(df_conciertos.sort_values('indice_popularidad_norm', ascending=False))

Recitales ordenados de mayor a menor popularidad en promedio:
          fecha  indice_popularidad_norm
138  2008-05-01                 0.885572
91   2004-10-10                 0.793305
160  2009-03-26                 0.655949
113  2007-05-06                 0.596631
145  2008-06-14                 0.573115
..          ...                      ...
5    1994-08-06                 0.284987
3    1993-07-09                 0.229025
2    1992-11-14                 0.224915
1    1991-11-09                 0.213641
0    1991-09-21                 0.195510

[178 rows x 2 columns]


In [136]:
# Filtrar fechas donde se hayan tocado más de 10 temas
fechas_mas_10 = df_super['fecha'].value_counts()
fechas_validas = fechas_mas_10[fechas_mas_10 > 15].index
df_super = df_super[df_super['fecha'].isin(fechas_validas)]
velez = df_super[df_super['fecha'] == '2004-05-22']
velez['indice_popularidad_norm'].median()

0.3058366526768369

In [137]:

# 3. Calcula el promedio de popularidad por concierto (por fecha)
df_conciertos = df_super.groupby(['fecha', 'lugar'])['indice_popularidad_norm'].mean().reset_index()
df_conciertos = df_conciertos.sort_values('indice_popularidad_norm', ascending=False)
df_conciertos = df_conciertos[df_conciertos['fecha'] > '2000-06-10']
df_conciertos = df_conciertos.sort_values(by= 'indice_popularidad_norm', ascending= False)
# 4. Ahora puedes ver los conciertos más "mainstream" y más "alternativos"
print("Recitales ordenados de mayor a menor popularidad en promedio:")
print(df_conciertos)

Recitales ordenados de mayor a menor popularidad en promedio:
          fecha                                              lugar  \
108  2007-11-17  Estadio Abierto del Parque de Mayo, San Juan, ...   
131  2008-11-15  Anfiteatro Mario del Tránsito Cocomarola, Corr...   
120  2008-05-02              Sala Assaig, Palma de Mallorca, Spain   
133  2008-11-22  Monumento al Obrero Petrolero, Caleta Olivia, ...   
112  2007-12-11               Estadio Ruca Che, Neuquén, Argentina   
..          ...                                                ...   
88   2006-04-26         Estadio Luna Park, Buenos Aires, Argentina   
34   2000-07-16  Estadio Obras Sanitarias, Buenos Aires, Argentina   
83   2005-10-01                 Estadio Unión, Santa Fe, Argentina   
85   2005-12-22      Estadio Boca Juniors, Buenos Aires, Argentina   
72   2004-05-22   Estadio Vélez Sarsfield, Buenos Aires, Argentina   

     indice_popularidad_norm  
108                 0.562223  
131                 0.558008  
120 

In [138]:
df_conciertos.to_excel('recitales_ordenados_por_popularidad.xlsx', index = False)

In [139]:
# Filtrar df_super para excluir canciones del álbum "Ineditos"
df_super_sin_ineditos = df_super[df_super['álbum'] != 'Ineditos']

# Calcular el promedio de popularidad por concierto (fecha y lugar) sin inéditos
df_conciertos_sin_ineditos = df_super_sin_ineditos.groupby(['fecha', 'lugar'])['indice_popularidad_norm'].mean().reset_index()
df_conciertos_sin_ineditos = df_conciertos_sin_ineditos[df_conciertos_sin_ineditos['fecha'] > '1998-02-10']
df_conciertos_sin_ineditos.sort_values(by= 'indice_popularidad_norm', ascending= False).tail(27)

,fecha,lugar,indice_popularidad_norm
135,2008-12-11,"Estadio Luna Park, Buenos Aires, Argentina",0.375416
73,2004-05-25,"Estadio Vélez Sarsfield, Buenos Aires, Argentina",0.374511
70,2004-01-30,"Patinódromo Municipal, Mar del Plata, Argentina",0.373043
27,1999-12-18,"Estadio Atlanta, Buenos Aires, Argentina",0.372201
93,2006-08-25,"Estadio Luna Park, Buenos Aires, Argentina",0.371743
26,1999-10-09,"Estadio All Boys, Buenos Aires, Argentina",0.370620
154,2025-05-24,"Parque de la Ciudad, Buenos Aires, Argentina",0.369988
86,2005-12-23,"Estadio Boca Juniors, Buenos Aires, Argentina",0.369546
90,2006-04-29,"Estadio Luna Park, Buenos Aires, Argentina",0.369146
92,2006-08-24,"Estadio Luna Park, Buenos Aires, Argentina",0.368913


In [144]:
df_conciertos_sin_ineditos[['Lugar', 'Localidad', 'País']] = df_conciertos_sin_ineditos['lugar'].str.split(',', expand=True)

df_conciertos_sin_ineditos[['Año', 'Mes', 'Día']] = df_conciertos_sin_ineditos['fecha'].str.split('-', expand=True)

In [ ]:
df_conciertos_sin_ineditos['ventas'] = df_conciertos_sin_ineditos['álbum'].map({
    "Ay Ay Ay": 60000,
    "3er Arco": 360000,
    "Chac Tu Chac": 60000,
    "Azul": 120000,
    "Verde Paisaje del Infierno": 120000,
    "Maquina de Sangre": 80000,
    "Civilizacion": 80000,
    "Ineditos": 80000
})

,fecha,lugar,indice_popularidad_norm,Lugar,Localidad,País,Año,Mes,Día
18,1998-05-17,"Parque Sarmiento, Buenos Aires, Argentina",0.354167,Parque Sarmiento,Buenos Aires,Argentina,1998,05,17
19,1998-08-22,Polideportivo Club de Gimnasia y Esgrima La Pl...,0.336848,Polideportivo Club de Gimnasia y Esgrima La Plata,La Plata,Argentina,1998,08,22
20,1998-10-24,"Estadio All Boys, Buenos Aires, Argentina",0.385213,Estadio All Boys,Buenos Aires,Argentina,1998,10,24
21,1999-01-23,"Autocine Villa Gesell, Villa Gesell, Argentina",0.361975,Autocine Villa Gesell,Villa Gesell,Argentina,1999,01,23
22,1999-05-07,"Estadio Obras Sanitarias, Buenos Aires, Argentina",0.367862,Estadio Obras Sanitarias,Buenos Aires,Argentina,1999,05,07
...,...,...,...,...,...,...,...,...,...
151,2025-04-26,"Hipódromo Parque de La Independencia, Rosario,...",0.466426,Hipódromo Parque de La Independencia,Rosario,Argentina,2025,04,26
152,2025-05-03,"Teatro Griego Frank Romero Day, Mendoza, Argen...",0.474947,Teatro Griego Frank Romero Day,Mendoza,Argentina,2025,05,03
153,2025-05-10,"Playón Mario Alberto Kempes, Córdoba, Argentina",0.486439,Playón Mario Alberto Kempes,Córdoba,Argentina,2025,05,10
154,2025-05-24,"Parque de la Ciudad, Buenos Aires, Argentina",0.369988,Parque de la Ciudad,Buenos Aires,Argentina,2025,05,24


In [142]:
# Crear un DataFrame de ejemplo
data = {'col1': ['valor1;valor2;valor3', 'valor4;valor5;valor6']}
df = pd.DataFrame(data)

# Dividir la columna 'col1' por el delimitador ';'
df[['col1', 'col2', 'col3']] = df['col1'].str.split(';', expand=True)

# El DataFrame ahora tiene las nuevas columnas
print(df)

     col1    col2    col3
0  valor1  valor2  valor3
1  valor4  valor5  valor6
